# Bird Sound Recordings Download
In this notebook, we will download and save audio recordings of birds from Xeno-canto. Xeno-canto is a website that offers a platform for sharing bird sound recordings from all over the world. We will download the top four most common bird species sound recordings in Xeno-canto from our respective countries. 

In [ ]:
#import the necessary libraries
import os
import re
import requests
from tqdm import tqdm
import ipywidgets as wg
import urllib.request, json
from collections import Counter
from pycountry import countries

In [ ]:
#Download and save the top 4 most common species in a given country

countries_list = list(countries) #access countries from the database object from pycountry
drop_down_display = [('Choose Country', -1)]

for indx, i in enumerate(countries_list):
    drop_down_display.append((i.name, indx)) #append tuples containing country names and index
                                            # for dropdown options

def countryOfInterest(country):
    """ Returns a list that contains dictionaries that contain links
        to download bird recordings, file names among other details.
    
    Args: 
        country: a string of the name of the country the recordings were done
    """
    recordings = []
    
    """the following three lines take care of countries with more than one name e.g. 'United Kingdom'
        to match the format required for advanced search in Xeno-canto"""
    
    link='https://www.xeno-canto.org/api/2/recordings?query=cnt:" "' 
    link = link.replace(' ', country)
    link = link.replace(' ', '%20')
    with urllib.request.urlopen(link) as url:
        data = json.loads(url.read().decode())
        
    pages = data['numPages'] # get the number of pages available for the query
    
    for page in range(pages): # iterate through the pages
        page  = '&page=' + str(page + 1)
        link='https://www.xeno-canto.org/api/2/recordings?query=cnt:" "' + page #specify the page in the search query
        link = link.replace(' ', country)
        link = link.replace(' ', '%20')
        with urllib.request.urlopen(link) as url:
            content = json.loads(url.read().decode())
            content = content['recordings']
            for dic in content:
                recordings.append(dic)
        
    
    return recordings


def fileSelect(country):
    """Returns a dictionary that comprise of links to download the 
    recordings with the the file names as the keys
    
    Args:
        country: a string of the country the recordings were taken
    
    """
    birds_dict = {}
    extension = '.mp3'
    recordings = countryOfInterest(country)
    birds = [name['en'] for name in recordings]
    num_birds = Counter(birds)
    most_frequent = num_birds.most_common(4)
    
    for bird in most_frequent: #this loop removes unknown birds from the most frequent bird species
        if bird[0] == 'Identity unknown':
            most_frequent = num_birds.most_common(5) #add the fifth most common bird species
            most_frequent.remove(bird) #remove the bird species labeled as unknown
            
    for bird in most_frequent:
        file_plus_links = {}
        for recording in recordings:
            if recording['en'] == bird[0] and recording['also'] == ['']:
                name = recording['file-name'][:-3]
                name = re.sub(r'[^\w\s]', '', name) #drop punctuations in the file name
                name = name + extension
                
                file_plus_links.update({name:'https:' + recording['file']})
                
        birds_dict.update({bird[0]:file_plus_links})

    return birds_dict


def fileDownload(country):
    """ Downloads and save the audio recordings in the specified path
    Args:
        country: a string of the country the recordings were taken
    """
    parent_dir = './xenocanto'
    birds_dict = fileSelect(country)
    for bird in birds_dict:
        audio_dir = os.path.join(parent_dir, bird)
        if not os.path.exists(audio_dir):
            os.makedirs(audio_dir)
        comp_file_list = list(birds_dict[bird])
        already_downloaded = os.listdir(audio_dir)
        comp_file_list = list(set(comp_file_list) - set(already_downloaded))
        for file in tqdm(comp_file_list): 
            url = birds_dict[bird][file]
            myfile = requests.get(url)
            open(os.path.join(audio_dir, file), 'wb').write(myfile.content)

def composeCountries(index):
    """Prepare list of countries to be indexed by the Dropdown
       Args: an integer returned by the dropdown menu 
    """
    
    list_of_counries = []
    
    for i in countries_list:
        list_of_counries.append(i.name)

    list_of_counries.append('Null')
    
    if list_of_counries[index] != 'Null':
        country  = list_of_counries[index]
        fileDownload(country)




## Dropdown to select country
In the cell that follows, we will develop a drop down from which one can select their country. On clicking the dropdown menu, a list of countries will appear. Select your country and the download will start. If your country does not appear on among the top displayed countries, type the first two letters of your country and it should appear. Select your country and the download will start. 

In [ ]:
#Develop a dropdown to select country of choice

try:
    drop_down = wg.Dropdown(options=drop_down_display,
               description='Country:',
               disabled=False)


    wg.interact(composeCountries, index=drop_down)
except Excepttion as e:
    print(e)
    pass